# Problema de Negócio - Churn em Telecomunicações

Este notebook ajuda a **entender o problema** antes de codar, definindo objetivos, métricas de sucesso e hipóteses.  
Use-o como guia de entrevistas com stakeholders.

## Contexto do Negócio
- **Objetivo**: reduzir *churn* (cancelamento de clientes) por meio de ações proativas.
- **Impacto**: cada cliente retido mantém receita recorrente (ARPU) e reduz custo de aquisição de novos clientes (CAC).
- **Definição de churn**: cliente que encerra o contrato no período de referência (ex.: próximo ciclo).

Operadoras perdem receita quando clientes cancelam o serviço (*churn*).  
Agir antes do cancelamento (retenção proativa) é mais barato do que adquirir novos clientes.

**Objetivo:** priorizar clientes com maior risco de churn para ações de retenção (desconto, upgrade, bônus), maximizando receita preservada dado um orçamento/capacidade operacional.

### Decisões-chave
- **Janela de previsão**: risco de churn no próximo ciclo (ex.: 30 dias).  
- **População-alvo**: clientes ativos, fora de período de carência.  
- **Estratégia operacional**: acionar o *top-k%* da lista ordenada pela probabilidade de churn (k definido por orçamento/capacidade).  
- **Métricas técnicas**: ROC-AUC, PR-AUC, Recall, Precision, Lift.  
- **Métricas de negócio**: clientes salvos, receita preservada, ROI.  

---

## ROI (Retorno sobre o Investimento)

O cálculo do ROI pode ser feito de forma simples e direta:

1. **Receita salva**  
   Quantos clientes o modelo ajudou a reter × valor médio mensal por cliente (ARPU).  

   $$
   \text{Receita Salva} = \text{Clientes Salvos} \times ARPU
   $$

   - **Clientes salvos**: número de clientes que teriam cancelado, mas permaneceram após a ação de retenção.  
   - **ARPU** (*Average Revenue Per User*): receita média por cliente, por mês.  

2. **Custo da ação**  
   Quantos clientes foram abordados × custo médio da ação de retenção.  

   $$
   \text{Custo} = \text{Clientes Abordados} \times Custo_{ação}
   $$

   - **Clientes abordados**: total de clientes que receberam a ação (todos os previstos pelo modelo).  
   - **Custo da ação**: valor gasto por cliente (ex.: desconto de R\$20, ligação do call center, bônus).  

3. **ROI**  
   A relação entre o ganho líquido e o custo:  

   $$
   ROI = \frac{\text{Receita Salva} - \text{Custo}}{\text{Custo}}
   $$

   - Se ROI > 0 → o projeto gera retorno positivo.  
   - Se ROI = 0 → retorno neutro (custo igual à receita preservada).  
   - Se ROI < 0 → prejuízo (custos superam a receita salva).  

---

## Exemplo rápido

- ARPU = **R\$ 100/mês**  
- Modelo sinalizou **1.000 clientes** → todos abordados  
- Desses, **200 foram realmente salvos** com a ação  
- Custo da ação = **R\$ 20 por cliente**  

**Cálculo:**
- Receita salva = 200 × 100 = **R\$ 20.000**  
- Custo = 1.000 × 20 = **R\$ 20.000**  
- ROI = (20.000 – 20.000) / 20.000 = **0 (empate)**  

Se o custo fosse menor (R\$ 10), o ROI seria positivo.  

---

## Riscos & Ética
- Evitar vieses (ex.: idade, renda) e documentar justificativas de uso.  
- Minimizar dados sensíveis; cumprir LGPD; explicar decisões quando possível.  

---

## Perguntas de Descoberta (para stakeholders)
1. Como a empresa define churn hoje (regras, datas, rótulo)?  
2. Qual o ARPU e a margem média por segmento? Existem produtos/planos com ARPU muito diferentes?  
3. Quais ações de retenção existem, seus custos e limites operacionais?  
4. Qual $k\%$ (ou orçamento) mensal é viável para abordagem?  
5. Há restrições (jurídicas, regulatórias) de uso de variáveis?  
6. Como será feita a integração (CRM, discador, helpdesk) e o monitoramento pós-deploy?  

---

## Métricas de Sucesso
- **Técnicas**: ROC-AUC, PR-AUC, Recall, Lift.  
- **De negócio**: clientes retidos, receita preservada, ROI líquido.  


# Sobre os Dados

## 📊 Descrição do Dataset — Telco Customer Churn

O **Telco Customer Churn Dataset** contém informações de clientes de uma empresa de telecomunicações.  
O objetivo é prever se um cliente irá **cancelar o serviço (Churn = Yes/No)** com base em variáveis demográficas, contratuais e de consumo de serviços.

### 🔑 Estrutura dos Dados
- **Total de registros**: 7.043 clientes  
- **Total de variáveis**: 21

### 📂 Variáveis disponíveis

**Informações demográficas**
- `customerID` → Identificador único do cliente
- `gender` → Gênero (Male/Female)
- `SeniorCitizen` → Indica se é idoso (1 = Sim, 0 = Não)
- `Partner` → Possui parceiro(a) (Yes/No)
- `Dependents` → Possui dependentes (Yes/No)

**Informações de permanência**
- `tenure` → Meses como cliente

**Serviços contratados**
- `PhoneService` → Serviço de telefonia (Yes/No)
- `MultipleLines` → Mais de uma linha telefônica (Yes/No/No phone service)
- `InternetService` → Tipo de internet (DSL, Fiber optic, No)
- `OnlineSecurity` → Serviço de segurança online (Yes/No/No internet service)
- `OnlineBackup` → Backup online (Yes/No/No internet service)
- `DeviceProtection` → Proteção de dispositivo (Yes/No/No internet service)
- `TechSupport` → Suporte técnico (Yes/No/No internet service)
- `StreamingTV` → TV por streaming (Yes/No/No internet service)
- `StreamingMovies` → Filmes por streaming (Yes/No/No internet service)

**Contrato e cobrança**
- `Contract` → Tipo de contrato (Month-to-month, One year, Two year)
- `PaperlessBilling` → Fatura digital (Yes/No)
- `PaymentMethod` → Método de pagamento (Eletrônico, Cheque, Cartão etc.)
- `MonthlyCharges` → Valor mensal cobrado
- `TotalCharges` → Valor total cobrado

**Variável alvo**
- `Churn` → Cliente cancelou o serviço? (Yes/No)

---

### 🎯 Objetivo do problema
Construir análises e modelos preditivos para identificar clientes com **alto risco de churn**, auxiliando a empresa em estratégias de retenção e tomada de decisão.


| Campo              | Descrição                      |          Tipo (bruto) | Papel                  | Exemplo            | Observações / Uso de Negócio                            |
| ------------------ | ------------------------------ | --------------------: | ---------------------- | ------------------ | ------------------------------------------------------- |
| `customerID`       | Identificador único do cliente |                string | ID (excluir do modelo) | `7590-VHVEG`       | Usado para *join*/rastreamento; não é feature.          |
| `gender`           | Gênero do titular              |            categórico | Feature                | `Female`/`Male`    | Normalmente pouco preditivo; avaliar importância.       |
| `SeniorCitizen`    | Se é idoso (1/0)               |        numérico (0/1) | Feature                | `1`                | Converter para categórico `Yes/No`; atenção a vieses.   |
| `Partner`          | Possui cônjuge/companheiro     |            categórico | Feature                | `Yes`              | Proxy de estabilidade/uso conjunto.                     |
| `Dependents`       | Possui dependentes             |            categórico | Feature                | `No`               | Pode correlacionar com fidelidade/planos família.       |
| `tenure`           | Meses de relacionamento        |              numérico | Feature                | `5`                | Forte indicador de risco: churn cai com maior tenure.   |
| `PhoneService`     | Possui telefonia               |            categórico | Feature                | `Yes`              | Sinaliza pacote de serviços.                            |
| `MultipleLines`    | Múltiplas linhas de telefonia  |            categórico | Feature                | `No phone service` | Tratar `No phone service` como categoria.               |
| `InternetService`  | Tipo de internet               |            categórico | Feature                | `Fiber optic`      | Fibra costuma ter dinâmica de churn distinta.           |
| `OnlineSecurity`   | Serviço de segurança online    |            categórico | Feature                | `No`               | Serviços adicionais indicam engajamento.                |
| `OnlineBackup`     | Backup online                  |            categórico | Feature                | `Yes`              | idem.                                                   |
| `DeviceProtection` | Proteção de dispositivo        |            categórico | Feature                | `No`               | idem.                                                   |
| `TechSupport`      | Suporte técnico                |            categórico | Feature                | `No`               | Insatisfação com suporte pode elevar churn.             |
| `StreamingTV`      | Streaming de TV                |            categórico | Feature                | `Yes`              | Itens de entretenimento aumentam *stickiness*.          |
| `StreamingMovies`  | Streaming de filmes            |            categórico | Feature                | `No`               | idem.                                                   |
| `Contract`         | Tipo de contrato               |            categórico | Feature                | `Month-to-month`   | **Muito preditivo**; mensal tende a maior churn.        |
| `PaperlessBilling` | Fatura eletrônica              |            categórico | Feature                | `Yes`              | Pode correlacionar com perfil digital e experiência.    |
| `PaymentMethod`    | Método de pagamento            |            categórico | Feature                | `Electronic check` | Cheque eletrônico costuma associar a maior churn.       |
| `MonthlyCharges`   | Cobrança mensal (R\$)          |              numérico | Feature                | `70.35`            | Normalizar; olhar relação não linear com churn.         |
| `TotalCharges`     | Cobrança total acumulada       | numérico (string/num) | Feature                | `1397.47`          | Converter p/ float; imputar vazios iniciais (tenure=0). |
| `Churn`            | Cancelou o serviço?            |   categórico (Yes/No) | **Alvo**               | `Yes`              | Converter para binário (1/0); estratificar split.       |


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("blastchar/telco-customer-churn")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/telco-customer-churn


In [3]:
import pandas as pd

df = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.shape

(7043, 21)

### Vamos olhar os dados, conhecer os dados, explorar os dados

In [4]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [7]:
df.InternetService.unique()

array(['DSL', 'Fiber optic', 'No'], dtype=object)

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
